In [1]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
import boto3
from decimal import Decimal
import time

In [2]:
with open(Path(r'../access.json'), 'r') as access_file:
    access = json.load(access_file)
    access_file.close()

In [8]:
data_path = Path(r'./S&P-sorted-data/').glob('*.json')

In [4]:
meta = Path(r'./meta-data/')

In [5]:
for i in data_path:
    meta_path = meta.joinpath(i.name)
    with open(meta_path, 'r') as meta_file:
        meta_data = json.load(meta_file)
        
        try:
            name = meta_data["Name"]
            print(name)
        except:
            name = None
        
        try:
            description = meta_data["Description"]
        except:
            description = None
        
        try:
            sector = meta_data["Sector"]
        except:
            sector = None

        try:
            industry = meta_data["Industry"]
        except:
            industry = None
        
        try:
            yearhigh = meta_data["52WeekHigh"]
            print(yearhigh)
        except:
            yearhigh = None

        try:
            yearlow = meta_data["52WeekLow"]
            print(yearlow)
        except:
            yearlow = None
        
        try:
            dividend = meta_data["DividendPerShare"]
        except:
            dividend = None



Agilent Technologies Inc
179.35
105.13
American Airlines Group Inc
26.09
10.93
Advance Auto Parts Inc
235.32
140.77
Apple Inc
157.26
111.63
AbbVie Inc
120.07
87.51
AmerisourceBergen Corporation
128.4
93.8
Abiomed Inc
376.99
254.41
Abbott Laboratories
129.53
103.72
Accenture plc
363.24
221.66
Adobe Inc
673.88
420.78
Analog Devices Inc
182.39
118.96
Archer-Daniels-Midland Company
68.89
45.29
Automatic Data Processing Inc
225.24
157.04
Autodesk Inc
344.39
241.46
Ameren Corporation
90.19
68.39
American Electric Power Company Inc
91.8
73.56
The AES Corporation
28.56
19.41
Aflac Incorporated
57.66
34.4
American International Group Inc
60.37
30.45
Assurant Inc
172.22
120.57
Arthur J. Gallagher & Co
168.5
105.3
Akamai Technologies Inc
121.11
92.64
Albemarle Corporation
260.99
92.15
Align Technology Inc
737.45
439.71
Alaska Air Group Inc
74.25
36.9
The Allstate Corporation
138.63
87.9
Allegion plc
148.31
100.12
Applied Materials Inc
145.48
61.61
Amcor plc
12.8
10.35
Advanced Micro Devices Inc
1

In [6]:
dynamodb = boto3.resource('dynamodb',region_name='us-east-2',aws_access_key_id=access['KEY_IAM_AWS'],aws_secret_access_key=access['SECRET_IAM_AWS'])

table = dynamodb.Table('tickers_sp')

In [7]:
def smaller_dict(d: dict()):
    list = [{k: v} for k, v in reversed(d.items())]
    return list

In [9]:
for i in data_path:
    meta_path = meta.joinpath(i.name)
    row_dict = {}
    with open(i, 'r') as json_file:
            print(i.stem)
            row_dict['ticker'] = i.stem
            
            data = json.load(json_file)
            
            
            row_dict['price'] = smaller_dict(data[i.stem]['price'])

            try:
                row_dict['pos_vals'] = data[i.stem]['pos_vals']
            except:
                row_dict['pos_vals'] = None

                print(f"skipping pos_vals for {i}")

            try:
                row_dict['neg_vals'] = data[i.stem]['neg_vals']
            except:
                row_dict['neg_vals'] = None
                
                print(f"skipping neg_vals for {i}")

            try:
                row_dict['dec_vals'] = data[i.stem]['decorrelated_vals']
            except:
                print(f"skipping neg_vals for {i}")
                row_dict['dec_vals'] = None

    json_file.close()
    
    with open(meta_path, 'r') as meta_file:
        meta_data = json.load(meta_file)
        try:
            row_dict['name'] = meta_data["Name"]
        except:
            name = None
            print(f"skipping name for {i}")

        try:
            row_dict['description'] = meta_data["Description"]
        except:
            row_dict['description'] = None
            print(f"skipping description for {i}")

        try:
            row_dict['industry'] = meta_data['Industry']
        except:
            row_dict['industry'] = None
            print(f"skipping industry for {i}")   

        try:
            row_dict['sector'] = meta_data['Sector']
        except:
            row_dict['sector'] = None
            print(f"skipping sector for {i}") 
        try:
            row_dict['yearhigh'] = meta_data["52WeekHigh"]
        except:
            row_dict['yearhigh'] = None
            print(f"skipping yearhigh for {i}") 

        try:
            row_dict['yearlow'] = meta_data["52WeekLow"]
        except:
            row_dict['yearlow'] = None
            print(f"skipping yearlow for {i}") 

        try:
            row_dict['dividend'] = meta_data["DividendPerShare"]
        except:
            row_dict['dividend'] = None   
            print(f"skipping dividend for {i}")

        meta_file.close()

    table.put_item(Item=json.loads(json.dumps(row_dict), parse_float=Decimal))
    time.sleep(1.5)


A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBWI
BBY
BDX
BEN
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
skipping name for S&P-sorted-data\BRK.B.json
skipping description for S&P-sorted-data\BRK.B.json
skipping industry for S&P-sorted-data\BRK.B.json
skipping sector for S&P-sorted-data\BRK.B.json
skipping yearhigh for S&P-sorted-data\BRK.B.json
skipping yearlow for S&P-sorted-data\BRK.B.json
skipping dividend for S&P-sorted-data\BRK.B.json
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CRL
CRM
CSCO
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CTXS
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
E